# Projeto: Previsão de Preços de Imóveis em Ames

### Importação das Bibliotecas Necessárias

In [ ]:
# Manipulação de Dados
import pathlib
import pickle
import numpy as np
import pandas as pd

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento e Modelagem
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configurações Gerais
%matplotlib inline
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

### Carregamento e Visualização dos Dados

In [ ]:
# Definir o caminho para o diretório de dados
DATA_DIR = pathlib.Path.cwd().parent / 'data'

# Construir o caminho completo para o arquivo
data_path = DATA_DIR / 'processed' / 'ames_with_correct_types.pkl'

# Verificar se o arquivo existe
if data_path.exists():
    print("Dados encontrados. Carregando o arquivo...")
    with open(data_path, 'rb') as file:
        data, continuous_variables, discrete_variables, ordinal_variables, categorical_variables = pickle.load(file)
    print("Dados carregados com sucesso.")
else:
    print("Erro: O arquivo não foi encontrado. Verifique o caminho e tente novamente.")

# Criar uma cópia do dataframe para manipulação
df = data.copy()

# Visualizar as primeiras linhas do dataframe
df.head()

### Análise Exploratória de Dados (EDA)

#### Informações Gerais

In [ ]:
# Informações sobre o dataframe
df.info()

#### Análise de Valores Ausentes

In [ ]:
# Calcular a porcentagem de valores ausentes em cada coluna
missing_percent = df.isnull().mean() * 100

# Exibir as colunas com valores ausentes
missing_percent[missing_percent > 0].sort_values(ascending=False)

#### Visualização de Distribuições

In [ ]:
# Visualizar a distribuição de 'SalePrice'
plt.figure(figsize=(8, 4))
sns.histplot(df['SalePrice'], kde=True)
plt.title('Distribuição de SalePrice')
plt.show()

#### Análise de Correlação

In [ ]:
# Matriz de correlação das variáveis numéricas
numeric_cols = continuous_variables + discrete_variables
corr_matrix = df[numeric_cols].corr()

# Heatmap da matriz de correlação
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

### Engenharia de Features

#### Tratamento de Valores Ausentes

Remover Colunas com Muitos Valores Ausentes

In [ ]:
# Identificar colunas com mais de 50% de valores ausentes
cols_to_drop = missing_percent[missing_percent > 50].index.tolist()
print(f'Colunas a serem removidas devido a muitos valores ausentes: {cols_to_drop}')

# Remover as colunas
df.drop(columns=cols_to_drop, inplace=True)

Imputação de Valores Ausentes

In [ ]:
# Imputação para variáveis numéricas
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

# Imputação para variáveis categóricas
cat_cols = df.select_dtypes(include=['category']).columns
for col in cat_cols:
    if df[col].isnull().any():
        df[col] = df[col].cat.add_categories(['Missing'])
        df[col] = df[col].fillna('Missing')

#### Transformação da Variável Alvo

In [ ]:
# Histograma de SalePrice antes da transformação
plt.figure(figsize=(8, 4))
sns.histplot(df['SalePrice'], kde=True)
plt.title('Distribuição de SalePrice (Antes da Transformação)')
plt.xlabel('SalePrice')
plt.ylabel('Frequência')
plt.show()

# Aplicar logaritmo natural
df['SalePrice'] = np.log(df['SalePrice'])

# Histograma de SalePrice após a transformação
plt.figure(figsize=(8, 4))
sns.histplot(df['SalePrice'], kde=True, color='green')
plt.title('Distribuição de SalePrice (Após a Transformação Logarítmica)')
plt.xlabel('Log(SalePrice)')
plt.ylabel('Frequência')
plt.show()

#### Criação de Novas Features

Total de Banheiros

In [ ]:
# Criar uma nova feature somando todos os banheiros
df['TotalBathrooms'] = (
    df['Bsmt.Full.Bath'] + (0.5 * df['Bsmt.Half.Bath']) +
    df['Full.Bath'] + (0.5 * df['Half.Bath'])
)

Idade da Casa

In [ ]:
# Calcular a idade da casa no ano da venda
df['HouseAge'] = df['Yr.Sold'] - df['Year.Built']
df['HouseAge'] = df['HouseAge'].apply(lambda x: x if x >= 0 else 0)

Área Total

In [ ]:
# Somar áreas relevantes
df['TotalSF'] = df['Total.Bsmt.SF'] + df['X1st.Flr.SF'] + df['X2nd.Flr.SF']

### Divisão dos Dados em Treino e Teste

In [ ]:
# Separar features e target
X = df.drop(columns='SalePrice')
y = df['SalePrice']

# Dividir os dados em treinamento e teste (75% treino, 25% teste)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

### Codificação de Variáveis Categóricas

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Identificar colunas categóricas e numéricas
categorical_cols = X_train.select_dtypes(include=['category']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Inicializar o OneHotEncoder com o parâmetro atualizado
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Ajustar o encoder nos dados de treinamento e transformar os dados de treinamento e teste
X_train_cat = ohe.fit_transform(X_train[categorical_cols])
X_test_cat = ohe.transform(X_test[categorical_cols])

# Obter os nomes das novas colunas
ohe_columns = ohe.get_feature_names_out(categorical_cols)

# Converter para DataFrame e manter o índice original
X_train_cat = pd.DataFrame(X_train_cat, columns=ohe_columns, index=X_train.index)
X_test_cat = pd.DataFrame(X_test_cat, columns=ohe_columns, index=X_test.index)

# Concatenar as features numéricas e categóricas
X_train = pd.concat([X_train[numerical_cols].reset_index(drop=True), X_train_cat.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test[numerical_cols].reset_index(drop=True), X_test_cat.reset_index(drop=True)], axis=1)

### Escalonamento das Features

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inicializar o escalonador
scaler = StandardScaler()

# Ajustar o escalonador nos dados de treinamento
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

# Transformar os dados de teste
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

### Construção e Treinamento de Múltiplos Modelos de Regressão

#### Regressão Linear Simples

In [ ]:
# Instanciar o modelo
lr_model = LinearRegression()

# Treinar o modelo
lr_model.fit(X_train, y_train)

# Prever no conjunto de teste
y_pred_lr = lr_model.predict(X_test)

# Calcular o RMSE
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print(f'RMSE Regressão Linear: {rmse_lr:.4f}')

#### Regressão Ridge

In [ ]:
# Definir a grade de hiperparâmetros
ridge_params = {'alpha': np.logspace(-3, 3, 7)}

# Instanciar o modelo
ridge = Ridge()

# Configurar o GridSearchCV
ridge_grid = GridSearchCV(
    ridge, ridge_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

# Treinar o modelo
ridge_grid.fit(X_train, y_train)

# Melhor modelo e hiperparâmetros
best_ridge = ridge_grid.best_estimator_
print(f'Melhor alpha para Ridge: {ridge_grid.best_params_['alpha']}')

# Previsões e RMSE
y_pred_ridge = best_ridge.predict(X_test)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
print(f'RMSE Regressão Ridge: {rmse_ridge:.4f}')

#### Regressão Lasso

In [ ]:
# Definir a grade de hiperparâmetros
lasso_params = {'alpha': np.logspace(-3, 1, 5)}

# Instanciar o modelo
lasso = Lasso(max_iter=10000)

# Configurar o GridSearchCV
lasso_grid = GridSearchCV(
    lasso, lasso_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

# Treinar o modelo
lasso_grid.fit(X_train, y_train)

# Melhor modelo e hiperparâmetros
best_lasso = lasso_grid.best_estimator_
print(f'Melhor alpha para Lasso: {lasso_grid.best_params_['alpha']}')

# Previsões e RMSE
y_pred_lasso = best_lasso.predict(X_test)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
print(f'RMSE Regressão Lasso: {rmse_lasso:.4f}')

#### Random Forest Regressor

In [ ]:
# Definir a grade de hiperparâmetros
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

# Instanciar o modelo
rf = RandomForestRegressor(random_state=42)

# Configurar o GridSearchCV
rf_grid = GridSearchCV(
    rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

# Treinar o modelo
rf_grid.fit(X_train, y_train)

# Melhor modelo e hiperparâmetros
best_rf = rf_grid.best_estimator_
print(f'Melhores hiperparâmetros para Random Forest: {rf_grid.best_params_}')

# Previsões e RMSE
y_pred_rf = best_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f'RMSE Random Forest: {rmse_rf:.4f}')

### Comparação dos Desempenhos dos Modelos

In [ ]:
# Criar um DataFrame com os resultados
results = pd.DataFrame({
    'Modelo': ['Regressão Linear', 'Regressão Ridge', 'Regressão Lasso', 'Random Forest'],
    'RMSE': [rmse_lr, rmse_ridge, rmse_lasso, rmse_rf]
})

# Ordenar por RMSE
results = results.sort_values('RMSE').reset_index(drop=True)
print(results)

#### Visualização dos Resultados

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='RMSE', y='Modelo', data=results, palette='viridis')
plt.title('Comparação dos Modelos')
plt.xlabel('RMSE')
plt.ylabel('Modelo')
plt.show()

### Avaliação do Modelo Selecionado

#### Métricas de Desempenho no Conjunto de Teste